### Importing packages

In [17]:
# Libraries
import time

# Reading in files
import pandas as pd
import numpy as np

# Progress bar
from tqdm.auto import tqdm

# Torch modules
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast as AutoTokenizer, AutoModel, AdamW, get_linear_schedule_with_warmup

# Lightning modules
import pytorch_lightning as pl
from torchmetrics.functional import accuracy, auroc
from torchmetrics import F1Score
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from torchmetrics.classification import MulticlassAccuracy, MulticlassF1Score, MulticlassPrecision, MulticlassRecall

# Split dataset/validation
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, multilabel_confusion_matrix
from sklearn.model_selection import StratifiedKFold

# Importing own functions 
from extract.importing_data import get_section

### Reading in data

In [18]:
df = pd.read_csv("../Data/Collated_dataset_for_scientific_papers.csv")

In [19]:
df = df[["Intro Concl", "Labels"]]

In [20]:
df.columns = ["string", "label"]

In [21]:
possible_labels = list(df['label'].unique())
possible_labels_num = list(range(0,len(possible_labels)))

In [22]:
len(possible_labels)

21

### Tokenizer 

In [23]:
MODEL_NAME = 'allenai/specter'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [24]:
mapping = zip(possible_labels, possible_labels_num)
label_to_idx = {label: num  for label, num in mapping}
mapping = zip(possible_labels, possible_labels_num)
idx_to_label = {num: label for label, num in mapping}

In [25]:
MAX_TOKEN_COUNT = 512
N_EPOCHS = 5
BATCH_SIZE = 25 # Changes: Edit the batch size here
KFOLD = 5

In [26]:
class SpecterDataset(Dataset):
    
    def __init__(self, data: pd.DataFrame, tokenizer: AutoTokenizer, max_token_len: int = MAX_TOKEN_COUNT, mapping = label_to_idx):
        self.tokenizer = tokenizer
        self.data = data
        self.max_token_len = max_token_len
        self.mapping = mapping
    
    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self, index: int):
    
        data_row = self.data.iloc[index]
        text = data_row["string"]
        labels = self.mapping[data_row["label"]]

        encoding = self.tokenizer.encode_plus(
          text,
          add_special_tokens=True,
          max_length=self.max_token_len,
          return_token_type_ids=False,
          padding="max_length",
          truncation=True,
          return_attention_mask=True,
          return_tensors='pt',
        )

        return dict(
          text=text,
          input_ids=encoding["input_ids"].flatten(),
          attention_mask=encoding["attention_mask"].flatten(),
          labels=labels
        )

In [27]:
class SpecterDataModule(pl.LightningDataModule):
    
    def __init__(self, df,
                 tokenizer,
                 k = 0,  # fold number
                 split_seed = 123,  # split needs to be always the same for correct cross validation
                 num_splits = KFOLD,
                 batch_size = BATCH_SIZE, 
                 max_token_len = MAX_TOKEN_COUNT,
                 num_workers = 0,
                 pin_memory = False):
        
        super().__init__()

        self.save_hyperparameters(logger=False)
    
    def setup(self, stage=None):

        # choose fold to train on
        kf = StratifiedKFold(n_splits=self.hparams.num_splits, shuffle=True, random_state=self.hparams.split_seed)
        all_splits = [k for k in kf.split(df, df.label)]
        train_indexes, val_indexes = all_splits[self.hparams.k]
        train_indexes, val_indexes = train_indexes.tolist(), val_indexes.tolist()

        self.data_train, self.data_val = df.iloc[train_indexes], df.iloc[val_indexes]
        
        self.train_dataset = SpecterDataset(
          self.data_train,
          self.hparams.tokenizer,
          self.hparams.max_token_len
        )
        self.val_dataset = SpecterDataset(
          self.data_val,
          self.hparams.tokenizer,
          self.hparams.max_token_len
        )
        
    def train_dataloader(self):
        return DataLoader(
          self.train_dataset,
          batch_size = self.hparams.batch_size,
          shuffle=True
        )
    
    def val_dataloader(self):
        return DataLoader(
          self.val_dataset,
          batch_size = self.hparams.batch_size
        )

### Modelling

In [28]:
class SpecterClassModel(pl.LightningModule):
    
    def __init__(self, n_classes: int, n_training_steps=None, n_warmup_steps=None):
    
        super().__init__()
        self.specter = AutoModel.from_pretrained(MODEL_NAME, return_dict=True)
        # Changes: Edit model architecture forward pass here
        self.classifier = nn.Linear(self.specter.config.hidden_size, n_classes)
        self.n_training_steps = n_training_steps
        self.n_warmup_steps = n_warmup_steps
        self.criterion = nn.CrossEntropyLoss()
        self.acc = MulticlassAccuracy(num_classes = 21, average = 'weighted') #measure performance based on weighted average
        self.f1 = MulticlassF1Score(num_classes = 21, average = 'weighted')
        self.prec = MulticlassPrecision(num_classes = 21, average = 'weighted')
        self.rec = MulticlassRecall(num_classes = 21, average = 'weighted')
        
        # Changes: Comment below code to remove freezing of the SPECTER embeddings
        for name, param in self.specter.named_parameters():
            if name.startswith('embeddings'):
                param.requires_grad = False
    
    
    def forward(self, input_ids, attention_mask, labels=None):
        output = self.specter(input_ids, attention_mask=attention_mask)
        # Changes: Edit model architecture forward pass here
        output = self.classifier(output.pooler_output)
        loss = 0
        if labels is not None:
            loss = self.criterion(output, labels)
            #acc = self.acc(output, labels)
            #f1 = self.f1(output, labels)
            #prec = self.prec(output, labels)
            #rec = self.rec(output, labels)
        return loss, output#, acc, f1, prec, rec

    def training_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        #loss, output, acc, f1, prec, rec = self(input_ids, attention_mask, labels)
        loss, output = self(input_ids, attention_mask, labels)
        self.log("train_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": output, "labels": labels}

    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels = batch["labels"]
        #loss, output, acc, f1, prec, rec = self(input_ids, attention_mask, labels)
        loss, output = self(input_ids, attention_mask, labels)
        self.log("val_loss", loss, prog_bar=True, logger=True)
        return {"loss": loss, "predictions": output, "labels": labels}
    
    def training_epoch_end(self, outputs):
        avg_loss = sum(output['loss'].item() for output in outputs) / len(outputs)
        
        predictions = torch.cat([output['predictions'] for output in outputs])
        labels = torch.cat([output['labels'] for output in outputs])
        
        acc = self.acc(predictions, labels)
        f1 = self.f1(predictions, labels)
        prec = self.prec(predictions, labels)
        rec = self.rec(predictions, labels)
        print("******Train epoch {} eval metrics: loss {:.8f}, f1 {:.4f} prec {:.4f} rec {:.4f}, acc {:.4f}".format(self.current_epoch, avg_loss, f1, prec, rec, acc))
        
        self.logger.experiment.add_scalars('loss', {'train': avg_loss}, self.current_epoch)   
    
    def validation_epoch_end(self, outputs):
        avg_loss = sum(output['loss'].item() for output in outputs) / len(outputs)

        predictions = torch.cat([output['predictions'] for output in outputs])
        labels = torch.cat([output['labels'] for output in outputs])
        
        acc = self.acc(predictions, labels)
        f1 = self.f1(predictions, labels)
        prec = self.prec(predictions, labels)
        rec = self.rec(predictions, labels)
        print("******Val epoch {} eval metrics: loss {:.8f}, f1 {:.4f} prec {:.4f} rec {:.4f}, acc {:.4f}".format(self.current_epoch, avg_loss, f1, prec, rec, acc))
        
            
        #For final output/Tensorboard visualisation
        self.log("Ignore/acc", acc, logger=True)
        self.log("Ignore/f1", f1, logger=True)
        self.log("Ignore/prec", prec, logger=True)
        self.log("Ignore/rec", rec, logger=True)
        
        #For Tensorboard visualisaion with Epoch as x axis
        self.logger.experiment.add_scalar('Collated/acc', acc, self.current_epoch)
        self.logger.experiment.add_scalar('Collated/f1', f1, self.current_epoch)
        self.logger.experiment.add_scalar('Collated/prec', prec, self.current_epoch)
        self.logger.experiment.add_scalar('Collated/rec', rec, self.current_epoch)
    
        self.logger.experiment.add_scalars('loss', {'val': avg_loss}, self.current_epoch)
    
    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=2e-5) # Changes: Edit the learning rate
        # Changes: Removed the scheduler
        scheduler = get_linear_schedule_with_warmup( # Changes: Edit the scheduler
          optimizer,
          num_warmup_steps=self.n_warmup_steps,
          num_training_steps=self.n_training_steps
        )
        
        # Changes: Edit the optimizer
        return dict(
          optimizer = optimizer,
          lr_scheduler = dict(
            scheduler = scheduler,
            interval = 'step'
          )
        )
        # Changes: Edit the optimizer
        #return dict(
        #  optimizer = optimizer
        #)

In [29]:
experiment = "FixedLearningRate(2e-5)"
time_now = time.strftime("%d_%m_%Y_%H_%M")

In [30]:
checkpoint_callback = ModelCheckpoint(
    dirpath = "checkpoints",
    filename = f"best-checkpoint-{experiment}",
    save_top_k = 1,
    verbose = True,
    monitor = "val_loss",
    mode = "min"
)


# early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [31]:
steps_per_epoch= round(len(df) * 0.8 // BATCH_SIZE)
total_training_steps = steps_per_epoch * N_EPOCHS
warmup_steps = total_training_steps // 5
warmup_steps, total_training_steps

(81, 405)

In [32]:
st_time = time.strftime("%d_%m_%Y_%H_%M")
result_acc_lst = []
result_f1_lst = []
result_prec_lst = []
result_recall_lst = []
nums_fold = 5
split_seed = 123

for k in range(nums_fold):
    data_module = SpecterDataModule(df, tokenizer, k = k)

    # here we train the model on given split...
    logger = TensorBoardLogger("lightning_logs", name = f"Specter_{experiment}_{time_now}_run{k}")
    model = SpecterClassModel(n_classes = 21, n_warmup_steps = warmup_steps, n_training_steps = total_training_steps)
    trainer = pl.Trainer(logger = logger, callbacks = [checkpoint_callback], max_epochs = N_EPOCHS, accelerator = "auto", devices=[1])
    trainer.fit(model, data_module)

    accuracy = trainer.callback_metrics['Ignore/acc'].item()
    f1 = trainer.callback_metrics['Ignore/f1'].item()
    precision = trainer.callback_metrics['Ignore/prec'].item()
    recall = trainer.callback_metrics['Ignore/rec'].item()
    
    result_acc_lst.append(accuracy)
    result_f1_lst.append(f1)
    result_prec_lst.append(precision)
    result_recall_lst.append(recall)
    break

average_val_acc_score = sum(result_acc_lst) / len(result_acc_lst)
average_val_f1_score = sum(result_f1_lst) / len(result_f1_lst)
average_val_prec_score = sum(result_prec_lst) / len(result_prec_lst)
average_val_recall_score = sum(result_recall_lst) / len(result_recall_lst)

ed_time = time.strftime("%d_%m_%Y_%H_%M")
print(st_time, ed_time)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
Missing logger folder: lightning_logs/Specter_FixedLearningRate(2e-5)_01_02_2023_09_18_run0
/home/yanxia/anaconda3/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /home/yanxia/projects/ACL_Anthology_Exploratory/Experiments/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]
/home/yanxia/anaconda3/lib/python3.9/site-p

Sanity Checking: 0it [00:00, ?it/s]

/home/yanxia/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


******Val epoch 0 eval metrics: loss 3.16727924, f1 0.0000 prec 0.0000 rec 0.0000, acc 0.0000


/home/yanxia/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 40 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


...

Validation: 0it [00:00, ?it/s]

Epoch 4, global step 410: 'val_loss' reached 1.16230 (best 1.16230), saving model to '/home/yanxia/projects/ACL_Anthology_Exploratory/Experiments/checkpoints/best-checkpoint-FixedLearningRate(2e-5)-v5.ckpt' as top 1


******Val epoch 4 eval metrics: loss 1.16677080, f1 0.6806 prec 0.6713 rec 0.7033, acc 0.7033
******Train epoch 4 eval metrics: loss 0.84222512, f1 0.7820 prec 0.8045 rec 0.8021, acc 0.8021


`Trainer.fit` stopped: `max_epochs=5` reached.


01_02_2023_09_18 01_02_2023_09_22


In [33]:
print(f"The average accuracy of the validation set across 5 folds is: {average_val_acc_score}")
print(f"The average F1 score of the validation set across 5 folds is: {average_val_f1_score}")
print(f"The average precision of the validation set across 5 folds is: {average_val_prec_score}")
print(f"The average recall of the validation set across 5 folds is: {average_val_recall_score}")

The average accuracy of the validation set across 5 folds is: 0.7033398747444153
The average F1 score of the validation set across 5 folds is: 0.6806449890136719
The average precision of the validation set across 5 folds is: 0.6712760925292969
The average recall of the validation set across 5 folds is: 0.7033398747444153


### Stop here for now We might want to take a look at creating a test set 